# Project 1: Kalman Filter 

**A note on this document**
This document is known as a Jupyter notebook; it allows text and executable code to coexist in a very easy-to-read format. Blocks can contain text or executable code. For blocks containing code, press `Shift + Enter`, `Ctrl+Enter`, or click the arrow on the block to run the code. Earlier blocks of code need to be run for the later blocks of code to work.

## Deliverable 1

**TODO**: Complete the `update` function inside the `KalmanFilter` class.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Any


class KalmanFilter:
    """Kalman Filter
    kf = KalmanFilter(A, B, C, Q, R)
    kf.set_init_values(x_init, P_init)
    xhat, P, K = kf.update(obs)

    Let A and C be, respectively, an n by n matirx and an m by n matrix. Then,
    xhat is the n by 1 state estimate, P will be the n by n state covariance matrix,
    and K will be the n by m Kalman gain.
    """

    def __init__(
        self,
        A: np.ndarray[Any, Any],
        B: np.ndarray[Any, Any],
        C: np.ndarray[Any, Any],
        Q: np.ndarray[Any, Any],
        R: np.ndarray[Any, Any],
    ) -> None:
        """Class constructor
        kf = KalmanFilter(A, B, C, Q, R) creates an LTI Kalman Filter.
        Args:
            A (_numpy array_): _System transition matrix_
            B (_numpy array): _Input matrix_
            C (_numpy array): _Observation matrix_
            Q (_numpy array_): _System covariance_
            R (_numpy array_): _Sbservation covariance_
        """

        if not isinstance(A, np.ndarray):
            raise TypeError("A must be a numpy ndarray.")
        if A.shape[0] != A.shape[1]:  # A must be an n by n matrix
            raise ValueError("A must be a square matrix (n x n).")

        if B.shape[0] != A.shape[0]:  # B must be an n by m matrix
            raise ValueError("B must be an n by m matrix (n x n).")

        if (
            Q.shape[0] != Q.shape[1] or Q.shape[0] != A.shape[0]
        ):  # Q must be an n by m matrix
            raise ValueError("Q must be a square matrix (n x n).")

        if A.shape[0] != C.shape[1]:  # C must be a k by n matrix
            raise ValueError("C must be a k by n matrix (k x n).")

        if (
            R.shape[0] != R.shape[1] or R.shape[0] != C.shape[0]
        ):  # C must be a k by n matrix
            raise ValueError("R must be a square matrix (k x k).")

        self.A = A
        self.B = B
        self.C = C
        self.Q = Q
        self.R = R
        self.xhat = np.array([])
        self.P = np.array([])
        self.K = np.array([])

    def set_init_values(self, x_init, Pinit) -> None:
        """_summary_

        Args:
            x_init (_numpy array_): _initial state estimate_
            Pinit (_numpy array_): _initial state covariance_
        """
        self.xhat = x_init
        self.P = Pinit

    def update(self, obs, u):
        """Update the optimal estimate
        xhat, P, K = kf.update(obs, u) updates the Kalman filter

        Args:
            obs (_numpy array_): _observation (measurement)_
            u (_numpy array_): _input_

        Returns:
            xhat (_numpy array_): n by 1 state estimate vector
            P (_numpy array_): n by n state covariance matrices,
            Pp (_numpy array_): n by n predicted state covariance matrices,
            K (_numpy array_): n by k Kalman gains

            where n is the state dimension, m is the input dimension, and k is the measurment dimesion.
        """

        """
        TODO: Implement the update function
        """
        # predict state estimate
        # xhat(k|k-1) = A*xhat(k-1|k-1) + B*u(k)
        xpred = 0

        # state prediction covariance matrix
        # P(k|k-1) = A*P(k-1|k-1)*A' + Q
        Ppred = 0

        # innovation
        # z_tilde(k) = z(k) - C*xhat(k|k-1)
        innovation = 0

        # innovation covariance
        # S = Cov[X(:,k) | y(:,1:k)] = C*P(k|k-1)*C' + R
        S = 0

        # Kalman gain
        # K(k) = P(k|k-1)*C'*inv(S)
        self.K = 0

        # update state estimate
        # xhat(k|k) = xhat(k|k-1) + K*z_tilde(k)
        self.xhat = 0

        # update state covariance matrix
        # P(k|k) = P(k|k-1) - K*C*P(k|k-1)
        self.P = 0

        return self.xhat, self.P, Ppred, self.K

## Deliverable 2

We will use the falling body example to estimate both the position and velocity of the object in motion. The measurements of the falling body consist of positions recorded over time, which may be subject to random noise following a Gaussian distribution with a zero mean and a variance of 4.

In [ ]:
y0 = 60
v0 = 20
g = -9.8067
Ts = 0.25

t = np.linspace(0, 6, 25)
mu, sigma = 0, 2
np.random.seed(20)
noise = np.random.normal(mu, sigma, len(t))

y_true = 1 / 2 * g * t**2 + v0 * t + y0
v_true = g * t + v0

# observations (measurements)
z = y_true + noise

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
fig.set
ax1.plot(t, y_true, "g", label="true")
ax1.plot(t, z, "bo", label="measured")
ax1.set_xlabel("time (sec)")
ax1.set_ylabel("height, y (m)", rotation=90)
ax1.grid(True)
ax1.legend(loc="lower left")

# numerical velocity
v_numeric = np.append(v0, np.diff(z) / Ts)

ax2.plot(t, v_true, "g", label="true")
ax2.plot(t, v_numeric, "bo", label="numerical")
ax2.set_xlabel("time (sec)")
ax2.set_ylabel("speed (m/s)")
ax2.grid(True)
ax2.legend(loc="lower left")

The equation of motion of a falling body in discrete time is given by
$$\begin{bmatrix} y_{k} \\ v_{k}\end{bmatrix} =\begin{bmatrix} 1 & T_s \\ 0 & 1\end{bmatrix} \begin{bmatrix} y_{k-1} \\ v_{k-1}\end{bmatrix} + \begin{bmatrix} 0 \\ T_s\end{bmatrix} g$$

Since we measure the position of the body, the equation of sensor dynamics is given by

$$z_{k} =\begin{bmatrix} 1 & 0 \end{bmatrix} \begin{bmatrix} y_{k} \\ v_{k}\end{bmatrix} $$


**TODO**: Select a system covariance value that results in a root mean square error (RMSE) of position estimates less than 2.0. Executing the next cell will give you RMSE values. 


In [ ]:
Ts = 0.25
g = np.array([[-9.81]])

A = np.array([[1, Ts], [0, 1]])
B = np.array([[0], [Ts]])
C = np.array([[1, 0]])

# The dimensions of {A,B,C} are important
print(f"Dimension of A: {A.shape}")
print(f"Dimension of B: {B.shape}")
print(f"Dimension of C: {C.shape}")

""" 
TODO: Select a system covariance that results in a RMSE of position estimates less than 2.0
"""
# Select a system variance
q_variance = 1000000
Q = np.eye(2) * q_variance

# measurment variance
R = np.array([[4]])

print(f"Dimension of Q: {Q.shape}")
print(f"Dimension of R: {R.shape}")

# TODO: initialize Kalman filter
kf = 0
x_init = 0
P_init = 0
kf.set_init_values(x_init, P_init)

# Data collection
xhats = []  # python list, not numpy array
state_covs = []  # python list, not numpy array
kalman_gains = []  # python list, not numpy array

for obs in z:
    xhat, state_cov, _, kalman_gain = kf.update(obs, g)

    xhats.append(xhat)
    state_covs.append(state_cov)
    kalman_gains.append(kalman_gain)

Now, it's time to assess the performance of the estimator. The following code will display the RMSE values. Ensure that the RMSE for the position estimate is under 2.0. Submit the printout on Gradescope.

In [ ]:
xhats = np.array(xhats)  # convert python list to numpy array
# reshape 25 x 2 X 1 array to 25 x 2 array
xhats = xhats.reshape((len(z), x_init.shape[0]))

kalman_gains = np.array(kalman_gains)  # convert python list to numpy arry
# reshape 25 x 2 X 1 array to 25 x 2 array
kalman_gains = kalman_gains.reshape((len(z), x_init.shape[0]))


# numerical velocity
v_numeric = np.append(v0, np.diff(z) / Ts)

mse_pos_obs = np.square(z - y_true).mean()
mse_pos_est = np.square(xhats[:, 0] - y_true).mean()

mse_vel_obs = np.square(v_numeric - v_true).mean()
mse_vel_est = np.square(xhats[:, 1] - v_true).mean()

print(f"RMSE of position measurements: {np.sqrt(mse_pos_obs)}")
print(f"RMSE of position estimates: {np.sqrt(mse_pos_est)}")
print(f"RMSE of velocity calculation: {np.sqrt(mse_vel_obs)}")
print(f"RMSE of velocity estimate: {np.sqrt(mse_vel_est)}")

**TODO**: 
1. Generate Plots:
    - Height and Speed: Create two separate plots, one for the height of the falling body over time and another for its speed.
    - Data to Include: Each plot should include the true values, measured (or numerical) values, and estimated values. This will help in comparing the performance of the estimator.
    - Legend: Make sure to add a legend to each plot to distinguish between the true, measured, and estimated data.
1. Upload to Gradescope:
    - Once the plots are generated, upload them to Gradescope as part of your assignment submission.
1. Comparison and Explanation:
    - Comparison: Provide a brief comparison between the measured (or numerical) values and the estimated values. Highlight any differences or similarities.
    - Explanation: Explain why the Kalman filter’s estimates are superior to the observations. This typically involves discussing how the Kalman filter reduces noise and provides more accurate estimates by combining predictions and measurements.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
fig.set

## Deliverable 3
Consider the following ground target tracking problem that we discussed in class.

In [ ]:
speed = 4.2  # m/s
radius = 120  # 120m
Ts = 1
time = np.arange(0, 180, Ts)
omega = speed / radius  # angular speed
x = np.sin(omega * time + np.sin(omega * time)) * radius
y = (np.cos(omega * time + np.cos(omega * time)) - np.cos(1)) * radius
path = np.row_stack((x, y))

mu, sigma = 0, 4
np.random.seed(17)

# z is a 2 by 180 vector
z = path + np.random.normal(mu, sigma, path.shape)
print(f"Dimension of z: {z.shape}")

plt.plot(path[0, :], path[1, :], "g")
plt.plot(z[0, :], z[1, :], "bo-")
plt.xlabel("latitude (m)")
plt.ylabel("longitude (m)")
plt.legend(["true", "measured"])

plt.grid(True)

The equation of motion of the ground target in discrete time is given by
$$\begin{bmatrix} x_{k+1} \\ y_{k+1} \\ \dot{x}_{k+1} \\ \dot{y}_{k+1}\end{bmatrix} =\begin{bmatrix} 1 & 0 & T_s & 0 \\ 0 & 1 &0 & T_s \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix} \begin{bmatrix} x_{k} \\ y_{k} \\ \dot{x}_{k} \\ \dot{y}_{k} \end{bmatrix} + \begin{bmatrix} 0 \\ 0 \\ a_{x,k} \\ a_{y,k} \end{bmatrix} $$
where $a_{x,k}$ and $a_{y,k}$ are zero mean Gaussian random noises representing system uncertainty.

Since we measure the position of the target, the equation of sensor dynamics is given by

$$z_{k} =\begin{bmatrix} x_{k} \\ y_{k} \end{bmatrix} + \mathbf{v} $$

where $\mathbf{v} \sim \mathcal{N}(\mathbf{0}, \mathbf{\sigma}_v)$ is zero mean Gaussian random noise representing the measurement noise.

Equivalently, it can be written in state space from as follows. 

$$z_{k} =\begin{bmatrix} 1 & 0 & 0 & 0 \\  0 & 1 & 0 & 0 \end{bmatrix} \begin{bmatrix} x_{k} \\ y_{k} \\ \dot{x}_{k} \\ \dot{y}_{k} \end{bmatrix} + \begin{bmatrix} v_{x, k} \\ v_{y, k} \end{bmatrix}$$

**TODO**: Complete the following code to find the best estimates of the trajectory of the ground target.

In [ ]:
Ts = 1

# TODO: Construct the A, B, C matrices
A = 0
B = 0
C = 0

# The dimensions of {A,B,C} are important
print(f"Dimension of A: {A.shape}")
print(f"Dimension of B: {B.shape}")
print(f"Dimension of C: {C.shape}")

# TODO: Select a system covariance that results in a RMSE of position estimates less than 3.0
q_variance = 0
Q = np.eye(4)  # 4 by 4 identity matrix
Q[2, 2] = q_variance
Q[3, 3] = q_variance

# TODO: Select a measurement covariance that results in a RMSE of position estimates less than 3.0
r_variance = 0
R = 0

# TODO: initialize Kalman filter
kf = 0

x_init = 0
P_init = 0

kf.set_init_values(x_init, P_init)

u = np.zeros_like(x_init)

# TODO: execute Kalman filter
# NOTE: z is a 2 by 180 vector





**TODO**: Complete the following code to print the RMSE values.

In [ ]:
xhats = np.array(xhats)  # convert python list to numpy array
# reshape 180 x 2 X 1 array to 180 x 2 array
xhats = xhats.reshape((z.shape[1], x_init.shape[0]))

# TODO: convert python list to numpy arry for kalman gains and reshape 180 x 4 X 2 array to 180 x 8 array
kalman_gains = 0
kalman_gains = 0

# numerical velocity
v_numeric = np.append(x_init[2:4], np.diff(z) / Ts)

# TODO: find the RMSE values


print(f"RMSE of position x measurements: {np.sqrt(mse_pos_obs_x)}")
print(f"RMSE of position x estimates: {np.sqrt(mse_pos_est_x)}")
print(f"RMSE of position y measurements: {np.sqrt(mse_pos_obs_y)}")
print(f"RMSE of position y estimates: {np.sqrt(mse_pos_est_y)}")

**TODO**: Generate a plot of the target trajectory estimate and submit it on Gradescope. You should include the true, measured, and estimated trajectories.

In [ ]:
fig, (ax1) = plt.subplots(1, 1)
fig.set